In [83]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [84]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

In [85]:
train_data = pd.read_csv('train.csv',index_col='SK_ID_CURR').sort_index(ascending=True)

x_train = train_data.drop(columns='TARGET')
y_train = train_data['TARGET'].astype('float')

In [86]:
numeric_columns = x_train._get_numeric_data().columns
categorical_columns = x_train.columns[~x_train.columns.isin(numeric_columns)]

In [87]:

numeric_transformer = Pipeline(steps= [
        ('imputer', SimpleImputer(strategy='median')),
        ('power_transformer', PowerTransformer(method='yeo-johnson')),
        ('mmsc', MinMaxScaler(feature_range=(0,1)))

])

categorical_transformer = Pipeline(steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ],remainder='passthrough')

# Create a pipeline with preprocessing and LogisticRegression
pipeline = Pipeline(steps= [
    ('preprocess',preprocessor)
])

# Fit the pipeline on training data
x_train = (pipeline.fit_transform(x_train)).toarray()
x_train = pd.DataFrame(x_train)



d:\Python\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [88]:
x_test = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\Data\\Data\\dseb63_final_project_DP_dataset\\dseb63_application_test.csv',index_col= 0)
x_test = x_test.set_index('SK_ID_CURR').sort_index(ascending=True)
submit = pd.DataFrame(x_test.index)

# update x_test
x_test['AMT_REQ_CREDIT_BUREAU_TOTAL_DAY'] = (
    x_test['AMT_REQ_CREDIT_BUREAU_HOUR'] / 24 +
    x_test['AMT_REQ_CREDIT_BUREAU_DAY'] +
    x_test['AMT_REQ_CREDIT_BUREAU_MON'] * 30 +
    x_test['AMT_REQ_CREDIT_BUREAU_YEAR'] * 365 +
    x_test['AMT_REQ_CREDIT_BUREAU_WEEK'] * 7 +
    x_test['AMT_REQ_CREDIT_BUREAU_QRT'] * 3 * 30
)
x_test['DAYS_BIRTH'] = x_test['DAYS_BIRTH'].apply(lambda x: int((x * -1) / 365))
x_test['DAYS_EMPLOYED'] = x_test['DAYS_EMPLOYED'].apply(lambda x: int((x * -1) / 365))
x_test = x_test.rename({'DAYS_BIRTH': 'AGE', 'DAYS_EMPLOYED' : 'YEARS_EMPLOYED'}, axis=1)
x_test = x_test[train_data.drop(columns='TARGET').columns]
x_test

,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,AGE,YEARS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_TOTAL_DAY
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,
5,Cash loans,F,1,540000.0,62698.5,Unaccompanied,Working,Higher education,Married,House / apartment,0.003122,49,3,Managers,3.0,Other,0.696321,0.586384,0.475850,30.0
13,Cash loans,F,1,72000.0,23508.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.015221,46,3,Sales staff,3.0,Self-employed,NaN,0.491704,0.372334,0.0
16,Cash loans,F,0,202500.0,29151.0,Unaccompanied,Commercial associate,Secondary / secondary special,Single / not married,Municipal apartment,0.018801,34,6,Sales staff,1.0,Business Entity Type 3,0.695093,0.564403,0.508287,1095.0
19,Cash loans,M,0,135000.0,16758.0,Family,State servant,Secondary / secondary special,Civil marriage,House / apartment,0.009657,30,7,Core staff,2.0,School,NaN,0.152479,0.456110,1460.0
25,Cash loans,F,2,90000.0,26217.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.006207,40,8,NaN,4.0,Kindergarten,0.517339,0.460314,0.504681,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307487,Cash loans,F,0,306000.0,24070.5,Unaccompanied,State servant,Higher education,Civil marriage,House / apartment,0.026392,47,13,High skill tech staff,2.0,University,NaN,0.728729,0.321735,0.0
307490,Cash loans,M,2,270000.0,46899.0,Family,State servant,Higher education,Married,House / apartment,0.025164,37,9,NaN,4.0,Police,0.657034,0.610301,0.520898,1125.0
307497,Cash loans,F,0,189000.0,42790.5,Unaccompanied,Commercial associate,Secondary / secondary special,Separated,House / apartment,0.018801,43,3,Laborers,1.0,Business Entity Type 3,NaN,0.618548,0.812823,365.0


#### Join

In [89]:
prev_app = pd.read_csv('train_data/Prev_app.csv')

In [90]:
merge_train = pd.merge(x_train, prev_app , right_index=True, left_index=True, how = 'inner')
merge_test = pd.merge(x_test, prev_app , right_index=True, left_index=True, how = 'inner')

In [91]:
merge_train

,0,1,2,3,4,5,6,7,8,9,...,Clothing_SELLER_INDUSTRY,Construction_SELLER_INDUSTRY,Auto technology_SELLER_INDUSTRY,Jewelry_SELLER_INDUSTRY,Tourism_SELLER_INDUSTRY,MLM partners_SELLER_INDUSTRY,middle_YIELD_GROUP,low_normal_YIELD_GROUP,low_action_YIELD_GROUP,high_YIELD_GROUP
0,0.000000,0.0,0.522974,0.634107,0.297757,0.132780,0.000000,0.258970,0.057295,0.208825,...,1,0,0,0,0,0,0,3,2,0
1,0.000000,0.0,0.412780,0.098344,0.704502,0.440244,0.203014,0.490473,0.512411,0.473255,...,0,0,0,0,0,0,0,1,0,1
2,0.883851,0.0,0.259524,0.703789,0.449760,0.208434,0.338574,0.418713,0.168365,0.335424,...,0,0,0,0,0,0,1,1,0,2
3,0.967084,0.0,0.365096,0.153881,0.531491,0.287985,0.439127,0.490473,0.029421,0.317285,...,0,0,0,0,0,0,2,1,0,0
4,0.000000,0.0,0.402942,0.502289,0.629301,0.132780,0.203014,0.490473,0.379745,0.464582,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246004,0.883851,0.0,0.514370,0.857296,0.511368,0.351154,0.338574,0.490473,0.828862,0.473255,...,0,0,0,0,0,0,0,0,0,1
246005,0.000000,0.0,0.239280,0.529987,0.847953,0.000000,0.000000,0.490473,0.013580,0.691610,...,0,0,0,1,0,0,1,0,0,0
246006,0.000000,0.0,0.599025,0.719493,0.228416,0.143805,0.203014,0.490473,0.879691,0.473255,...,0,0,0,0,0,0,0,1,0,2
246007,0.000000,0.0,0.429234,0.529987,0.899642,0.000000,0.203014,0.490473,0.292889,0.580146,...,0,0,0,0,0,0,2,1,0,0


### Tuning Parameter

### Predict

In [92]:
model = LogisticRegression(random_state= 0)
model.fit(x_train,y_train)
x_test = pipeline.transform(x_test)
y_pred = model.predict_proba(x_test)[:, 1]


d:\Python\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [93]:
submit['TARGET'] = y_pred
submit = submit.set_index('SK_ID_CURR')
submit.to_csv('../Submission.csv')
